In [4]:
from torchvision.io.image import read_image
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image

import os
perfil = 'raquelcanas'
# Ruta de la carpeta que contiene las imágenes y subcarpetas
ruta_carpeta = f'images\{perfil}'
output_dir = f'bbox'

# Step 1: Initialize model with the best available weights
weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Iterar sobre los elementos de la carpeta
for raiz, carpetas, archivos in os.walk(ruta_carpeta):
    for archivo in archivos:
        # Filtrar solo los archivos de imagen (puedes agregar más extensiones según tus necesidades)
        extensiones_permitidas = [".jpg", ".jpeg", ".png", ".gif"]
        if any(archivo.endswith(ext) for ext in extensiones_permitidas):
            # Imprimir la ruta completa del archivo de imagen
            ruta_completa = os.path.join(raiz, archivo)
            img = read_image(ruta_completa)
           # print(f"Imagen encontrada: {ruta_completa}")
            # Step 3: Apply inference preprocessing transforms
            batch = [preprocess(img)]

            # Step 4: Use the model and visualize the prediction

            prediction = model(batch)[0]
            labels = [weights.meta["categories"][i] for i in prediction["labels"]]

            # Filter predictions to include only "person" labels
            person_indices = [i for i, label in enumerate(labels) if label == "person"]
            filtered_boxes = prediction["boxes"][person_indices]
            filtered_labels = [labels[i] for i in person_indices]

            # Draw bounding boxes only for persons
            box = draw_bounding_boxes(img, boxes=filtered_boxes,
                                    labels=filtered_labels,
                                    colors="red",
                                    width=4, font_size=30)
            im = to_pil_image(box.detach())
            os.makedirs(f'{output_dir}{raiz}', exist_ok=True)
            im.save(f'{output_dir}{ruta_completa}')